In [1]:
import s3fs
import xarray as xr
import ujson
import dask
import numpy as np
from xmitgcm import llcreader

#from xarray.core.combine import _nested_combine
from functools import partial
from fronts.llc.slurp import write_xr


def _multi_file_closer(closers):
    for closer in closers:
        closer()

In [13]:
# You (the user) should edit this for your use case. 
# Define which face or faces you are using
#face_range = range(10,11)
face_range = range(0,13)
#possible values are from 0 to 12

#define which iterations you want. 
start_from = 0 # 0 is the first record of Eta, ~1180 is the first record for wind
length_in_hours = 12
time_step_in_hours = 12 # the minimum timestep between files is one hour


start_iter = 10368 + start_from*144
end_iter = 10368 + start_from*144 + length_in_hours*144
iter_range = np.arange(start_iter,end_iter,time_step_in_hours*144)

get_Eta_files = True

In [14]:
endpoint_url = 'https://mghp.osn.xsede.org'
fs = s3fs.S3FileSystem(anon=True,
    client_kwargs={'endpoint_url': endpoint_url}
)


if (get_Eta_files):
    filelist = ['cnh-bucket-1/llc_surf/kerchunk_files/llc4320_Eta-U-V-W-Theta-Salt_f' + str(var1) + '_k0_iter_' + 
            str(var) + '.json' for var1 in face_range for var in iter_range]
else:
    filelist = ['cnh-bucket-1/llc_wind/kerchunk_files/llc4320_KPPhbl-PhiBot-oceTAUX-oceTAUY-SIarea_f' + str(var1) + '_k0_iter_' + 
            str(var) + '.json' for var1 in face_range for var in iter_range]


mapper = [fs.open(file, mode='rb') for file in filelist]


reflist = [ujson.load(mapper1)for mapper1 in mapper]

open_ = dask.delayed(xr.open_dataset)
getattr_ = dask.delayed(getattr)

dict_list = [{"storage_options": {"fo": p,"remote_protocol": "s3",
    "remote_options": {"client_kwargs":{'endpoint_url': endpoint_url}, "anon": True}},"consolidated": False} for p in reflist]
datasets = [open_("reference://",engine="zarr",backend_kwargs=p,chunks={'i':720, 'j':720}) for p in dict_list]
closers = [getattr_(ds, "_close") for ds in datasets]
datasets, closers = dask.compute(datasets, closers)


ds = xr.combine_by_coords([dataset for dataset in datasets],compat="override", coords='minimal', combine_attrs='override')


for ds1 in datasets:
            ds1.close()

ds.set_close(partial(_multi_file_closer, closers))

In [21]:
str(ds.time.values)[2:21].replace(':','_')

'2011-09-13T00_00_00'

# xmitgcm

In [9]:
model = llcreader.ECCOPortalLLC4320Model()
tstep_hr = 144  # Time steps per hour

# Get dataset
iter_step = tstep_hr*12
ds = model.get_dataset(varnames=['Eta'],
                               k_levels=[0], type='latlon',
                               iter_step=iter_step)
tsize = ds.time.size
print("Model is ready")

/home/xavier/miniconda3/envs/os_313/lib/python3.13/site-packages/xmitgcm/llcreader/llcmodel.py:322: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  data = np.arange(ifac * coords.dims[vname])
/home/xavier/miniconda3/envs/os_313/lib/python3.13/site-packages/xmitgcm/llcreader/llcmodel.py:324: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  data = np.arange(jfac * coords.dims[vname])


Model is ready


In [10]:
tsize

860

In [12]:
ds.time

<xarray.DataArray 'time' (time: 860)> Size: 7kB
array(['2011-09-13T00:00:00.000000000', '2011-09-13T12:00:00.000000000',
       '2011-09-14T00:00:00.000000000', ..., '2012-11-14T12:00:00.000000000',
       '2012-11-15T00:00:00.000000000', '2012-11-15T12:00:00.000000000'],
      shape=(860,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 7kB 2011-09-13 ... 2012-11-15T12:00:00